#<font color=blue>**YOLO를 통한 버클리 데이터셋 학습**</font>

##<font color=green>**버클리 데이터 업로드 진행**</font>

####<font color=red>구글 드라이브 업로드 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

####<font color=red>업로드한 BDD_data.zip 압축풀기

In [ ]:
!unzip /content/drive/MyDrive/DataSet.zip

####<font color=red>각각의 데이터 개수 확인

In [ ]:
import os

images_train=os.listdir('/content/DataSet/images/train/')
images_val=os.listdir('/content/DataSet/images/val/')

labels_train=os.listdir('/content/DataSet/labels/train/')
labels_val=os.listdir('/content/DataSet/labels/val/')


print(f'train images 개수:{len(images_train)}')
print(f'validation images 개수:{len(images_val)}')

print("")
print(f'train labels 개수:{len(labels_train)}')
print(f'validation labels 개수:{len(labels_val)}')




##<font color=green>**git의 YOLOv5 설치**</font>

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

##<font color=green>**학습 시작**</font>

In [ ]:
# Train YOLOv5s on custom_data for 20 epochs
!python train.py --img 640 --batch 16 --epochs 10 --data custom_data.yaml --weights yolov5s.pt --cache

In [ ]:
!ls runs/train/exp

####<font color=red>학습된 Loss 그래프 확인

In [ ]:
from IPython.display import Image as Display # to display images
Display(filename='runs/train/exp/results.png', width=1080)

####<font color=red>텐서보드를 통해 Loss 그래프 확인

In [ ]:
%reload_ext tensorboard
# TensorBoard 준비하기
%load_ext tensorboard
%tensorboard --logdir ./runs

####<font color=red>원하는 이미지로 테스트</font>

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source ../car.png

####<font color=red>대회 결과 다운로드</font>
Loss 그래프

yaml 파일

마지막까지 학습되어 도출된 모델

학습중 베스트 모델

In [ ]:
from google.colab import files
files.download('/content/yolov5/runs/train/exp/results.png')
files.download('/content/yolov5/data/custom_data.yaml')
files.download('/content/yolov5/runs/train/exp/weights/last.pt')
files.download('/content/yolov5/runs/train/exp/weights/best.pt')
